In [49]:
# packages
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os, os.path
import matplotlib.patches as patches
%matplotlib tk

In [51]:
# find the max brightness values indices if repeats
def indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

In [122]:
# function for determining star brightness over time
def brightness_fun(coords, imgs):
    # define rounded coordinates
    rounded_coords = (round(coords[0]),round(coords[1]))

    # create star_brightness list
    star_bright = []

    # LOOP over all images
    for i in range(0,len(imgs)):

        # find the brightness values in the area
        coord_brightness = []
        brightness = []
        big_square = 10
        for j in range(-big_square,big_square):
            for k in range(-big_square,big_square):
                coord_brightness.append([(rounded_coords[0]+k, rounded_coords[1]+j), np.array(imgs[i])[rounded_coords[1]+j][rounded_coords[0]+k]])
                brightness.append(np.array(imgs[i])[rounded_coords[1]+j][rounded_coords[0]+k])

        # find the maximum brightness
        max_brightness = max(coord_brightness, key=lambda v: v[1])
    
        # define new coordinate reference point for next image
        rounded_coords = max_brightness[0]

        # find corners of star
        x_coord = []
        y_coord = []
        for j in indices(brightness, max_brightness[1]):
            x_coord.append(coord_brightness[j][0][1])
            y_coord.append(coord_brightness[j][0][0])
        star_corners = [(min(x_coord),min(y_coord)), (max(x_coord),max(y_coord))]
    
        # define small square around star
        small_r = 3
        small_square = [(star_corners[0][0]-small_r,star_corners[0][1]-small_r), (star_corners[1][0]+small_r,star_corners[1][1]+small_r)]

        # make list of all brightness values in the small square
        small_square_brightness = []
        for j in range(small_square[0][0],small_square[1][0]+1):
            for k in range(small_square[0][1],small_square[1][1]+1):
                small_square_brightness.append([(k,j), np.array(imgs[i])[j][k]])

        # average the brightness values over the small square
        cum_bright = 0
        for j in range(0,len(small_square_brightness)):
            cum_bright += small_square_brightness[j][1]

        # make a list of average brightness values for the star over time
        avg_bright = cum_bright/(len(small_square_brightness)+1)
        star_bright.append(avg_bright)       

    # return the list of average brightness values for the star over time
    return star_bright

In [288]:
# create a list of images in the folder
imgs = []
path = "star-images"

# unpack them in order
for i in range(0,len(os.listdir(path))-1):
    filename = 'image-' + str(i) + '.jpg'
    full_path = os.path.join(path, filename)

    imgs.append(Image.open(full_path).convert('L'))

In [290]:
# select stars (transit star then reference star)
plt.imshow(imgs[0])
click_coords = plt.ginput(2)
plt.close() 

# generate and label brightness lists for both stars
transit_star = brightness_fun(click_coords[0], imgs)
reference_star = brightness_fun(click_coords[1], imgs)

# define scaled brightness array
scaled_transit = []
for i in range(0,len(transit_star)):
    scaled_transit.append(100*(transit_star[i])/reference_star[i])

In [292]:
# set up plot
fig, axs = plt.subplots(1,2,layout='constrained',figsize=(9,3.15))

# create x_pts
x_pts = np.linspace(-0.75,0.75,len(scaled_transit))

#plot top graph
axs[0].plot(x_pts, transit_star, label='Transit Star')
axs[0].plot(x_pts, reference_star, label='Reference Star')
axs[0].set_ylim(100)
axs[0].legend()
axs[0].set_ylabel('Stellar Brightness [mean pixel value]')
axs[0].set_xlabel('Normalised Transit Duration')

#plot bottom graph
axs[1].plot(x_pts, scaled_transit, 'o')
axs[1].set_ylabel('Normalised Stellar Brightness [%]')
axs[1].set_xlabel('Normalised Transit Duration')

plt.show()